In [1]:
import numpy as np
from tqdm import tqdm
import os
import json
from torch.utils.data import Dataset, Sampler
import torch.utils.data as data

/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get features, labels and timecodes

In [4]:
features =[]
labels = []
timeLabel = []

path = '/scratch/users/mdelabrassinne/Database/'


for root, dirs, files in tqdm(os.walk(path)):
    #print(files)
    for file in files:
            
        if file.endswith("Labels-v2.json"):
            
            p = os.path.dirname(os.path.join(root, file))
            dire = os.path.basename(p).replace('_', ' ')
            p = os.path.join(os.path.dirname(p), dire)
            
            for r, d, fs in os.walk(p):
                for f in fs:
                    if f.endswith("baidu_soccer_embeddings.npy"):
                        p_current = os.path.join(r, f)
                        features.append(p_current)
                        #print(p_current)

            
            jsonPath =(os.path.join(root, file))
            #print(jsonPath)
            f = open(jsonPath)
            d = json.load(f)
            
            #print(jsonPath)
            
            try:
                #print('test')
                lab = [d['annotations'][i]['label'] for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '1']         # 1fps, don't know if keep 
                time = [60 * int(d['annotations'][i]['gameTime'].split(':')[0][-2:]) + int(d['annotations'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '1']
                #print([d['annotations'][i]['gameTime'] for i in range(len(d['annotations']))])
                
                #print(time)
                
                labels.append(lab)
                timeLabel.append(time)
                
                lab = [d['annotations'][i]['label'] for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '2']         # 1fps, don't know if keep 
                time = [60 * int(d['annotations'][i]['gameTime'].split(':')[0][-2:]) + int(d['annotations'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '2']
                
                #print(time)
                labels.append(lab)
                timeLabel.append(time)
                
            except:
                #print(d)
                lab = [d['predictions'][i]['label'] for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '1']         # 1fps, don't know if keep 
                time = [60 * int(d['predictions'][i]['gameTime'].split(':')[0][-2:]) + int(d['predictions'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '1']
                
                labels.append(lab)
                timeLabel.append(time)
                
                lab = [d['predictions'][i]['label'] for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '2']         # 1fps, don't know if keep 
                time = [60 * int(d['predictions'][i]['gameTime'].split(':')[0][-2:]) + int(d['predictions'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '2']
                
                labels.append(lab)
                timeLabel.append(time)
            

            
data_l = list(zip(features, labels, timeLabel))

98714it [00:58, 1699.55it/s]


### Function for the transformation of the ground truths

In [5]:
cL = ['Corner', 'Goal', 'Penalty', 'Kick-off', 'Yellow card', 'Red card', 'Yellow->red card']

def getTrueCoef(label, timeCodes, T, nbClasses, r_c = 3, f = 1, r_d = 6):
    C = []      #confidence as a 2D np array instead of a list of tupes

    res_conf = np.zeros((T, nbClasses))
    res_disp = np.zeros((T, nbClasses))
    
    dictClasses = {}

    for c in range(nbClasses):
        dictClasses[c] = []


    for i in range(len(timeCodes)):        #loop over obtained labels
        t = timeCodes[i]
        c = label[i]
        
        if(c in cL):                           # focus on wanted classes
            c = cL.index(label[i])

            tmin = max(0, t - r_c * f)
            tmax = min(t + r_c * f, T-1)
            inds = np.arange(tmin, tmax+1)
            res_conf[inds, c] = 1
            
            dictClasses[c].append(t)
            
        if(t > T):
            print('\n\naie => t = {} -- T = {}\n\n'.format(t, T))
            
        if(i >= 2700):
            print("possible issue ")

    for c in range(nbClasses):
        ind = np.array(dictClasses[c])
        if(len(ind) > 0):
            for t in range(T):

                try:
                    closestInd = np.argmin(np.abs(ind - t))         #take the one that is the most on the left if draw
                except:
                    print(ind)
                    print(t)
                    print(len(ind))

                if(np.abs(t - ind[closestInd]) <= r_d * f):
                    res_disp[t, c] = t - ind[closestInd]
            
        
    return res_conf, res_disp

test_C, test_D = getTrueCoef(data_l[0][1],data_l[0][2], np.load(data_l[0][0]).shape[0], len(cL) )

print(np.where(test_C != 0))
print(np.where(test_D != 0))

print(test_C.shape)
print(test_D.shape)

(array([   0,    1,    2,    3,  179,  180,  181,  182,  183,  184,  185,
        289,  290,  291,  292,  293,  294,  295,  664,  665,  666,  667,
        668,  669,  670,  787,  788,  789,  790,  791,  792,  793,  840,
        841,  842,  843,  844,  845,  846,  877,  878,  879,  880,  881,
        882,  883,  968,  969,  970,  971,  972,  973,  974, 1028, 1029,
       1030, 1031, 1032, 1033, 1034, 2018, 2019, 2020, 2021, 2022, 2023,
       2024, 2405, 2406, 2407, 2408, 2409, 2410, 2411, 2542, 2543, 2544,
       2545, 2546, 2547, 2548]), array([3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0]))
(array([   1,    2,    3,    4,    5,    6,  176,  177,  178,  179,  180,
        181,  183,  184,  185,  186,  187,  188,  286,  287,  288,  289,
        290,  291,  293,  294,  295

### Dataset, dataloader et test

In [34]:
cL = ['Corner', 'Goal', 'Penalty', 'Kick-off', 'Yellow card', 'Red card', 'Yellow->red card']

class BaiduDataset(Dataset):    #modifiable
    """
    Class to prepare the data for a neural network in pyTorch. The database is composed of 3 tables:
    - Video: to save the video information
    - Sequence: to save the sequence information labelled with the soccer actions and record the sound of the sequence
    - Image: to save the image information of the sequence
    """
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform
        self.seq_length = 112
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        feat_path = self.data_list[idx][0]       # path towards Baidu features
        label_list = self.data_list[idx][1]      # list of labels
        time_list = self.data_list[idx][2]       # list of timecodes

        features = np.load(feat_path)
        
        indStart = np.ceil(np.random.uniform(0, features.shape[0]-self.seq_length))
        ind = np.array([True if (i >= indStart and i < indStart + self.seq_length) else False for i in range(features.shape[0])])
        
        coef_c, coef_d = getTrueCoef(label_list, time_list, features.shape[0], len(cL) )
        
        timeInds = np.array(time_list)
        print(timeInds.shape)
        zeArray = np.zeros(self.seq_length - timeInds.shape[0])
        timeInds = np.hstack((timeInds, zeArray))
        #print(features.shape)
        #print(coef_c.shape)
        #print(coef_d.shape)
        #print(features[ind,:].shape)
        #print(coef_c[ind,:].shape)
        #print(coef_d[ind,:].shape)
        
        #print("\n")
        return (features[ind,:], coef_c[ind, :], coef_d[ind, :])   #not of equal size => issue

In [35]:
dataLearning = BaiduDataset(data_l)
loaderLearning = data.DataLoader(dataLearning, batch_size = 20, shuffle= True, num_workers = 1)
feat, lab, t = next(iter(loaderLearning))

(109,)
(109,)
(134,)
(142,)
(65,)
(103,)
(74,)
(139,)


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_3430781/2275669072.py", line 32, in __getitem__
    zeArray = np.zeros(self.seq_length - timeInds.shape[0])
ValueError: negative dimensions are not allowed


### Model

In [25]:
a = np.zeros(5)
i = [2,3]
a[i] = 1
print(a)

[0. 0. 1. 1. 0.]


### Training

### test loss

In [8]:
import torch.nn as nn

In [9]:
def huberLoss(pred, gTruth, treshold=0.5):
    res = pred - gTruth
    
    abs_res = np.abs(res)
    quadratic_loss = 0.5 * np.square(res)
    linear_loss = treshold * (abs_res - 0.5 * treshold)
    
    loss = np.where(abs_res <= treshold, quadratic_loss, linear_loss)
    return loss

In [10]:
def confLoss2(c_preds, c_truths, T):
    if(c_preds.shape[0] != T):
        c_preds = c_preds.reshape(-1)

    if(c_truths.shape[0] != T):
        c_truths = c_truths.reshape(-1)

    criterion = nn.CrossEntropyLoss()       

    return criterion(c_preds, c_truths)


def dispLoss2(d_preds, d_truths, T):
    if(d_preds.shape[0] != T):
        d_preds = d_preds.reshape(-1)

    if(d_truths.shape[0] != T):
        d_truths = d_truths.reshape(-1)

    return huberLoss(d_preds, d_truths)


def loss(c_preds, d_preds, c_truths, d_truths, T):
    l = confLoss2(c_preds, c_truths, T)
    

In [12]:
test_c = np.random.randn(2700, 7)
test_d = np.random.randn(2700, 7)

true_c = np.random.randn(2700, 7)
true_d = np.random.randn(2700, 7)